## Parallel Aguirregabiria

In [3]:
using Plots; pyplot()

# Parallel computing settings
PROCESS_NUMBER = 4
if nprocs() <= 1
    # Create new processes:
    addprocs(PROCESS_NUMBER)
end

4-element Array{Int64,1}:
 2
 3
 4
 5

In [27]:
function generate_simplex_3dims(n_per_dim::Integer=20) 

    simplex = [[x, y, 1-x-y] for x in linspace(0,1,n_per_dim) for y in linspace(0,1,n_per_dim) if x+y <= 1.0]

    return hcat(simplex...)'

end

# Numerical parameters
lambdas      = SharedArray{Float64}(generate_simplex_3dims(20))
n_price_grid = 20
min_price    = 0.5
max_price    = 1.5
price_grid   = SharedArray{Float64}(collect(linspace(min_price, max_price, n_price_grid)))

# Load core model code everywhere
@everywhere begin
    
# Model parameters
    
betas_transition = [-4, -1.7, -1.2]
sigma_eps = 0.5
alpha     = 1.0
c         = 0.5
delta     = 0.9
    
using FastGaussQuadrature
    
dmd_transition_fs(x,y,z) = 1/sqrt(2*pi*sigma_eps^2)*exp.(-(x-(alpha+betas_transition.*log(y))).^2./(2*sigma_eps^2))
    
function belief(new_state, action, old_state, lambda_weights)
    return dot(dmd_transition_fs(new_state, action, old_state), lambda_weights)
end 

function update_lambdas(new_state, action, old_state, old_lambdas)
    den = dot(old_lambdas, dmd_transition_fs(new_state, action, old_state))
    return (dmd_transition_fs(new_state, action, old_state)/den).*old_lambdas 
end
    
function rescale_demand(d, beta_l, price)
    # Rescales demand to use Gauss-Hermite 
    # collocation points.
    mu = alpha + beta_l*log.(price)
    
    return (sqrt(2)*sigma_eps*d + mu)
end
                    
function period_return(price_grid, lambdas)
    constant_part = (price_grid - c) * exp(alpha + (sigma_eps^2)/2)
    summation     = exp.(log.(price_grid)*betas_transition') * lambdas
                    
    #constant_part = (p-const.c) * np.e ** const.α * np.e ** ((const.σ_ɛ ** 2) / 2)
    #summation = np.dot(np.e**(betas_transition*np.log(p[:, np.newaxis])), lambdas)

    return (summation.*constant_part)

end
                
function myopic_price(lambdas)
                    
    elasticity = dot(lambdas, betas_transition)
    @assert elasticity < -1.0
    
    return c/(1+(1/elasticity))
                    
    #elasticity = np.dot(lambdas, betas_transition) #-2.2
    #assert elasticity < -1.0
    #return const.c / (1 + (1/elasticity))
                    
end
                    
function tripolate(V, x, y)
    # Triangular (linear) interpolation:
    # 0<=|x|<=1, 0<=|y|<=1 and N parition points
    # of [0,1] for x, y
    N  = size(V, 1)-1
    xf = Int(floor(N*x) - floor(x))
    yf = Int(floor(N*y) - floor(y))
    x_ = (N*x - xf)
    y_ = (N*y - yf)
    if min(xf, yf) == 0 && max(xf, yf) <= 1
        x_ = 1-x_
        y_ = 1-y_
    end
    if xf + yf <= 1
        return V[xf+2,yf+1]*(1-x_) + V[xf+1,yf+2]*(1-y_) - V[xf+1,yf+1]*(1-x_-y_)
    else
        return V[xf+2,yf+1]*(x_) + V[xf+1,yf+2]*(y_) + V[xf+1,yf+1]*(1-x_-y_)
    end 
end

function interpV(simplex, V)
    N_simplex = Int((sqrt(8*size(lambdas, 1)+1)-1)/2)
    augment_V = zeros(N_simplex, N_simplex)
    k = 1
    for i in 1:N_simplex
        for j in 1:(N_simplex+1-i)
            augment_V[i,j] = V[k]
            k += 1
        end
    end

    interpolate_V(x) = tripolate(augment_V, x[1], x[2])

    return interpolate_V, augment_V
end
    
end

In [32]:
policy = SharedArray{Float64}(zeros(size(lambdas, 1)))
T_V    = SharedArray{Float64}(zeros(size(lambdas, 1)))

function pll_bellman_operator(Vguess, T_V, policy, price_grid, lambda_simplex)

    # 1. Go over grid of state space
    # 2. Write objective (present return + delta*eOfV)
    # 3. Find optimal p on that objective
    # 4. Write optimal p and value function on that point in the grid
    
    GHx, GHw = gausshermite(25)
    GHx = SharedArray{Float64}(GHx)
    GHw = SharedArray{Float64}(GHw)
    
    
    
    @sync begin 
    ###=== PARALLEL CODE BLOCK ===###   
        
    @parallel for index in 1:size(lambda_simplex, 1)
        
        lw = lambda_simplex[index, :]
        
        # Compute period returns
        R = period_return(price_grid, lw)
        
        # Compute the expectation integral of V0
        E0fV_p = zeros(price_grid)
        
        @inbounds for (i, price) in enumerate(price_grid)
            # Gauss-Hermite integration using pre-computed
            # collocations and weights.
            sum_over_each_lambda = 0.0
            @inbounds for (l, beta_l) in enumerate(betas_transition)
                @inbounds for (k, hermite_point) in enumerate(GHx)
                    rescaled_remand = rescale_demand(hermite_point, beta_l, price)
                    new_lw          = update_lambdas(rescaled_remand, price, ~, lw)
                    #v_value = f(new_lambdas_val)
                    #sum_over_each_lambda += V(new_lw)*GHw[k]*lw[l]
                    sum_over_each_lambda += tripolate(Vguess,new_lw[1],new_lw[2])*GHw[k]*lw[l]
                end
            end
            E0fV_p[i] = (1/sqrt(pi) * sum_over_each_lambda)
        end
        
        #println(E0fV_p)
            
        # Maximize value function
        objective_vals = R + (delta *  E0fV_p)
            
        T_V[index], ind_max = findmax(objective_vals)                       
        policy[index] = price_grid[ind_max]

    end
        
    ###=== PARALLEL CODE BLOCK ===###    
    end

    # Interpolate
    interp_T_V = interpV(lambda_simplex, T_V)

    return interp_T_V, policy
    
end

# Execute parallel VFI
function pll_compute_fixed_point(V, price_grid, lambda_simplex; error_tol=1e-5, max_iter=50, verbose=true, skip=10)

    iterate = 1
    error = error_tol + 1
    
    policy = similar(price_grid)
    error  = Inf

    while iterate <= max_iter && error > error_tol
        if verbose && (mod(iterate, skip) == 0)
            tic()
        end

        new_V, policy = adv_bellman_operator(V, price_grid, lambda_simplex)
                        
        # Compute error over simplex:
        error = 0
        for i in 1:size(lambda_simplex,1)
            error += abs(V(lambda_simplex[i,:])-new_V(lambda_simplex[i,:]))
        end

        V = new_V

        if verbose && (mod(iterate, skip) == 0)
            println(@sprintf("Computed iterate %d with error %.4f", iterate, error))
            print(" !-- ")
            toc()
        end
                        
        iterate += 1                
                        
    end

    return V, policy, error

end

pll_compute_fixed_point (generic function with 1 method)

In [35]:
@everywhere function V_0(x) 
    optimal_price = myopic_price(x)
    return period_return(optimal_price, x)
end

tmp, V0 = interpV(lambdas, zeros(size(lambdas,1)))
V0 = SharedArray{Float64}(V0)

@time res_1 = pll_bellman_operator(V0, T_V, policy, price_grid, lambdas)

 11.345710 seconds (3.88 k allocations: 234.875 KiB)


((interpolate_V, [1.89353 1.87524 … 1.59066 1.58133; 1.82589 1.8076 … 1.55615 0.0; … ; 2.50178 2.51163 … 0.0 0.0; 2.59611 0.0 … 0.0 0.0]), [1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5  …  0.657895, 0.657895, 0.657895, 0.657895, 0.657895, 0.657895, 0.657895, 0.657895, 0.657895, 0.657895])